<a href="https://colab.research.google.com/github/changyong93/Natural-language-processing-with-chat-bot/blob/main/%EB%94%A5%EB%9F%AC%EB%8B%9D%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC_%EC%9E%85%EB%AC%B8(11_7_RNN%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%B6%84%EB%A5%98_%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%87%BC%ED%95%91_%EB%A6%AC%EB%B7%B0_%EA%B0%90%EC%84%B1_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 네이버 쇼핑 리뷰 감성 분류하기(Naver Shopping Review Sentiment Analysis)
---
- 다운로드 링크 : https://github.com/bab2min/corpus/tree/master/sentiment

In [ ]:
#konlpy 설치
!pip install konlpy

#mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## 네이버 쇼핑 리뷰 데이터에 대한 이해와 전처리

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")

In [ ]:
total_data = pd.read_table("ratings_total.txt", names = ['rating','reviews'])
total_data.head()

In [ ]:
print("전체 데이터 개수: ", len(total_data))

### train & test 데이터 나누기

In [ ]:
total_data['label'] = (total_data.rating >= 3).astype(int)
# total_data['label'] = np.select([total_data.rating > 3], [1],0)

total_data.head()

In [ ]:
print(total_data.shape)
print(total_data.nunique())

In [ ]:
#중복 제거
print("총 샘플 수(중복 제거 전)", len(total_data))
total_data = total_data.drop_duplicates(subset = ["reviews"]).copy()
print("총 샘플 수(중복 제거 후)", len(total_data))

In [ ]:
#null
total_data.isnull().sum()

In [ ]:
train_data, test_data = train_test_split(total_data, test_size = 0.25, random_state = 42)
train_data.shape, test_data.shape

### 분포 확인

In [ ]:
train_data.label.value_counts().plot.bar()
train_data.label.value_counts()

### 데이터 정제

In [ ]:
train_data["reviews"] = train_data["reviews"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data["reviews"] = train_data["reviews"].replace("^ +","")
train_data["reviews"] = train_data["reviews"].replace("", np.nan)
train_data = train_data.dropna(how = 'any').copy()

test_data = test_data.drop_duplicates(subset = ["reviews"]).copy()
test_data["reviews"] = test_data["reviews"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
test_data["reviews"] = test_data["reviews"].replace("^ +", "")
test_data["reviews"] = test_data["reviews"].replace("", np.nan)
test_data = test_data.dropna(how = 'any').copy()


In [ ]:
print("테스트 데이터 개수: ", len(test_data))

### 토큰화

In [ ]:
mecab = Mecab()

In [ ]:
print(mecab.morphs('와 이런 것도 상품이라고 차라리 내가 만드는 게 나을 뻔'))

In [ ]:
#stopwords 지정
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']

In [ ]:
train_data["tokenized"] = train_data["reviews"].apply(mecab.morphs)
train_data["tokenized"] = train_data["tokenized"].apply(lambda x : [word for word in x if word not in stopwords])

test_data["tokenized"] = test_data["reviews"].apply(mecab.morphs)
test_data["tokenized"] = test_data["tokenized"].apply(lambda x : [word for word in x if word not in stopwords])

### 단어 길이와 분포 확인

In [ ]:
negative = np.hstack(train_data.loc[train_data.label==0,"tokenized"].values)
positive = np.hstack(train_data.loc[train_data.label==1,"tokenized"].values)

In [ ]:
negative_word_count = Counter(negative)
print(negative_word_count.most_common(20))

In [ ]:
positive_word_count = Counter(positive)
print(positive_word_count.most_common(20))

In [ ]:
bin_num = 20
fig,(ax1,ax2) = plt.subplots(1,2,figsize = (12,5))
# [len(text) for text in train_data.loc[train_data.label ==0, "tokenized"]]
text_len = train_data.loc[train_data.label ==0, "tokenized"].apply(len)
ax1.hist(text_len, bins = bin_num, color = 'red')
ax1.set_xlabel("length of samples")
ax1.set_ylabel("number of samples")
ax1.set_title("Negative Reviews")
print("부정 리뷰 평균 길이: ",round(np.mean(text_len),3))
print("부정 리뷰 최대 길이: ",max(text_len))

text_len = train_data.loc[train_data.label ==1, "tokenized"].apply(len)
ax2.hist(text_len, bins = bin_num, color = 'blue')
ax2.set_xlabel("length of samples")
ax2.set_ylabel("number of samples")
ax2.set_title("Positive Reviews")
print("긍정 리뷰 평균 길이: ",round(np.mean(text_len),3))
print("긍정 리뷰 최대 길이: ",max(text_len))

plt.show()

In [ ]:
X_train = train_data.tokenized.values
y_train = train_data.label.values

X_test = test_data.tokenized.values
y_test = test_data.label.values

### 정수인코딩

In [ ]:
X_train.shape, X_test.shape

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
len(tokenizer.word_index)

In [ ]:
#단어 빈도수가 1개인 것 제거
threshold = 2
total_cnt = len(tokenizer.word_index)
rare_cnt = 0 # 전체 단어 중 빈도수가 2 미만인 개수 카운트
total_freq = 0 #전체 단어의 빈도수
rare_freq = 0 #rare_cnt에 해당하는 단어의 빈도수

for key,value in tokenizer.word_counts.items():
  total_freq += value

  if value < threshold:
    rare_cnt += 1
    rare_freq += value

print("전체 단어 집합(vocabulary) 크기: ", total_cnt)
print("전체 단어 중 등장 빈도가 1개인 단어의 개수: ", rare_cnt)
print(f"전체 단어에서 희귀 단어의 비율: {rare_cnt / total_cnt * 100  :.3f}%")
print(f"전체 단어 빈도에서 희귀 단어 빈도 비율: {rare_freq / total_freq * 100 :.3f}%")

In [ ]:
#등장 빈도수 1 이하인 단어 제외
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2
vocab_size = total_cnt - rare_cnt + 2
print("단어 집합의 크기: ",vocab_size)

In [ ]:
#정수인코딩
tokenizer = Tokenizer(num_words = vocab_size, oov_token = "OOV")
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
print(X_train[:3])
print("*"*100)
print(X_test[:3])

### 패딩

In [ ]:
print("리뷰 최대 길이: ", max([len(s) for s in X_train]))
print("리뷰 평균 길이: ", np.mean([len(s) for s in X_train]))

plt.hist([len(s) for s in X_train], bins = 50)
plt.xlabel("length of samples")
plt.ylabel("number of samples")
plt.show()

In [ ]:
max_len = max([len(s) for s in X_train])

for lens in range(1,max_len+1):
  cnt = sum([1 for idx in range(len(X_train)) if len(X_train[idx]) <= lens])
  print(f"전체 샘플 중 전체 길이가 {lens}이하인 샘플의 비율: {cnt / len(X_train) * 100 :.3f}%")


In [ ]:
max_len = 80
X_train = pad_sequences(sequences=X_train, maxlen = max_len)
X_test = pad_sequences(sequences=X_test, maxlen = max_len)

## GRU로 네이버 쇼핑 리뷰 감성 분류

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
model = Sequential()
model.add(Embedding(input_dim = vocab_size, output_dim = 100))
model.add(GRU(units = 128))
model.add(Dense(units = 1, activation = 'sigmoid'))

es = EarlyStopping(monitor = 'val_loss', patience = 4, verbose = 1, mode = "min")
mc = ModelCheckpoint(filepath = "base_model.h5", monitor='val_acc',mode = "max",save_best_only=True, verbose = 1)

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy',metrics = ['acc'])
history = model.fit(x = X_train, y = y_train, batch_size = 60, epochs = 15, validation_split=0.2, callbacks = [es, mc])

In [ ]:
loaded_model = load_model("base_model.h5")
print(f"테스트 정확도: {loaded_model.evaluate(X_test,y_test)[1]:.4f}")

## 리뷰 예측

In [ ]:
def sentiment_predict(new_sentence):
  new_sentence = mecab.morphs(new_sentence) #토큰화
  new_sentence = [word for word in new_sentence if word not in stopwords] #불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) #정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len)
  score = float(loaded_model.predict(pad_new))
  if score > 0.5:
    print(f"{score*100:.2f}% 확률로 긍정 리뷰입니다.")
  else:
    print(f"{(1-score)*100:.2f}확률로 확률로 긍정 리뷰입니다.")

In [ ]:
sentiment_predict('이 상품 진짜 좋아요... 저는 강추합니다. 대박')

In [ ]:
sentiment_predict('진짜 배송도 늦고 개짜증나네요. 뭐 이런 걸 상품이라고 만듬?')

In [ ]:
sentiment_predict('판매자님... 너무 짱이에요.. 대박나삼')

In [ ]:
sentiment_predict('ㅁㄴㅇㄻㄴㅇㄻㄴㅇ리뷰쓰기도 귀찮아')